___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.researchgate.net/publication/331474581/figure/fig1/AS:1131188974567432@1646707876257/A-unified-deep-learning-framework-for-time-series-classification.png" width="500px" height="300px" />


# <font color= #8A0829> Clasificación de series de tiempo usando algoritmos de ML.</font>

<Strong> Objetivos </Strong>
- Cómo cargar y evaluar algoritmos de aprendizaje automático no lineales y de conjunto en la versión con ingeniería de características del conjunto de datos de reconocimiento de actividad.
- Cómo cargar y evaluar algoritmos de aprendizaje automático en los datos de señal sin procesar del conjunto de datos de reconocimiento de actividades.

> Referencias:
    > - Capítulo 23 de [Deep Learning for Time Series Forecasting: Predict the Future with MLPs, CNNs and LSTMs in Python](https://machinelearningmastery.com/deep-learning-for-time-series-forecasting/)
___

# 1. Modelado de datos de ingeniería de características

En esta sección, desarrollaremos el código para cargar la versión del conjunto de datos con ingeniería de características y evaluaremos un conjunto de algoritmos de aprendizaje automático no lineal, incluido el SVM utilizado en el artículo original. **El objetivo es lograr al menos un 89% de precisión en el conjunto de datos de prueba**. Los resultados de los métodos que utilizan la versión con características modificadas del conjunto de datos proporcionan una línea de base para cualquier método desarrollado para la versión de datos sin procesar desarrollado en sesiones previas.

## 1.0 Selección de características 


Las características seleccionadas para esta base de datos proceden de las señales brutas triaxiales del acelerómetro y el giroscopio tAcc-XYZ y tGyro-XYZ. Estas señales en el dominio del tiempo (prefijo "t" para denotar el tiempo) se capturaron a una frecuencia constante de 50 Hz. A continuación, se filtraron utilizando un filtro de mediana y un filtro Butterworth de paso bajo de 3er orden con una frecuencia de esquina de 20 Hz para eliminar el ruido. Del mismo modo, la señal de aceleración se separó en señales de aceleración del cuerpo y de la gravedad (tBodyAcc-XYZ y tGravityAcc-XYZ) utilizando otro filtro Butterworth de paso bajo con una frecuencia de esquina de 0,3 Hz. 

Posteriormente, la aceleración lineal del cuerpo y la velocidad angular se derivaron en el tiempo para obtener las señales Jerk (tBodyAccJerk-XYZ y tBodyGyroJerk-XYZ). También se calculó la magnitud de estas señales tridimensionales utilizando la norma euclidiana (tBodyAccMag, tGravityAccMag, tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag). 

Por último, se aplicó una transformada rápida de Fourier (FFT) a algunas de estas señales para obtener fBodyAcc-XYZ, fBodyAccJerk-XYZ, fBodyGyro-XYZ, fBodyAccJerkMag, fBodyGyroMag, fBodyGyroJerkMag. (Nótese la "f" para indicar las señales en el dominio de la frecuencia). 

Estas señales se utilizaron para estimar las variables del vector de características de cada patrón:  
-XYZ' se utiliza para denotar señales de 3 ejes en las direcciones X, Y y Z.

        tBodyAcc-XYZ
        tGravityAcc-XYZ
        tBodyAccJerk-XYZ
        tBodyGyro-XYZ
        tBodyGyroJerk-XYZ
        tBodyAccMag
        tGravityAccMag
        tBodyAccJerkMag
        tBodyGyroMag
        tBodyGyroJerkMag
        fBodyAcc-XYZ
        fBodyAccJerk-XYZ
        fBodyGyro-XYZ
        fBodyAccMag
        fBodyAccJerkMag
        fBodyGyroMag
        fBodyGyroJerkMag

El conjunto de variables que se estimaron a partir de estas señales son: 

        mean(): Valor medio
        std(): Desviación estándar
        mad(): Desviación absoluta mediana 
        max(): Mayor valor de la matriz
        min(): Valor más pequeño de la matriz
        sma(): Área de magnitud de la señal
        energía(): Medida de energía. Suma de los cuadrados dividida por el número de valores. 
        iqr(): Rango intercuartílico 
        entropía(): Entropía de la señal
        arCoeff(): Coeficientes de autorregresión con orden Burg igual a 4
        correlation(): Coeficiente de correlación entre dos señales
        maxInds(): índice de la componente de frecuencia de mayor magnitud
        mediaFreq(): Media ponderada de las componentes de frecuencia para obtener una frecuencia media
        skewness(): asimetría de la señal en el dominio de la frecuencia 
        curtosis(): curtosis de la señal en el dominio de la frecuencia 
        bandasEnergía(): Energía de un intervalo de frecuencia dentro de los 64 bins de la FFT de cada ventana.
        ángulo(): Ángulo entre to vectores.

Vectores adicionales obtenidos promediando las señales en una muestra de ventana de señal. Se utilizan en la variable angle():

        gravityMean
        tBodyAccMean
        tBodyAccJerkMean
        tBodyGyroMean
        tBodyGyroJerkMean

La lista completa de variables de cada vector de características está disponible en 'features.txt'

## 1.1 Carga de datos

En esta ocasión cargaremos los datos que cuentan con la información de los datos con la ingeniería de características que cuentan con los datos de entrada $X$ y salida $y$. Para ello vamos a cargar los siguientes archivos

- UCI HAR Dataset/train/X_train.txt
- UCI HAR Dataset/train/y_train.txt
- UCI HAR Dataset/test/X_test.txt
- UCI HAR Dataset/test/y_test.txt

Para este proposito creamos las siguientes funciones:

In [1]:
# Carga de librerías a utilizar
import pandas as pd
import numpy as np

In [4]:
# cargar un solo archivo como una matriz numpy
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values


# cargar un grupo de conjuntos de datos, como train o test
def load_dataset_group(group, prefix=''):
    # load input data
    X = load_file(prefix + group + '/X_'+group+'.txt')
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# carga el conjunto de datos, devuelve el los datos de entrenamiento y prueba X y y
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
    print(testX.shape, testy.shape)
    # flatten y
    trainy, testy = trainy[:,0], testy[:,0]
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [5]:
# load dataset
trainX, trainy, testX, testy = load_dataset()

(7352, 561) (7352, 1)
(2947, 561) (2947, 1)
(7352, 561) (7352,) (2947, 561) (2947,)


In [6]:
np.unique(trainy)

array([1, 2, 3, 4, 5, 6])

## 1.2 Difinición de modelos

A continuación, podemos definir una lista de modelos de aprendizaje automático para evaluarlos en este problema. Evaluaremos los modelos utilizando configuraciones predeterminadas. En este punto, no buscamos configuraciones óptimas de estos modelos, sino una idea general del rendimiento de los modelos sofisticados con configuraciones predeterminadas en este problema. Evaluaremos un conjunto diverso de algoritmos de aprendizaje automático no lineales y por grupos, en concreto.

Los algoritmos que utlizaremos serán los siguientes:

**Nonlinear Algorithms:**
- k-Nearest Neighbors
- Classification and Regression Tree
- Support Vector Machine
- Naive Bayes

**Ensemble Algorithms:**
- Bagged Decision Trees
- Random Forest
- Extra Trees
- Gradient Boosting Machine

Para ellos vamos importar los paquetes con los modelos correspondientes y crearemos una función que instancie cada uno de los modelos dentro de un diccionario.

In [7]:
# Importar modelos a utilizar
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
# crear un dict de modelos estándar para evaluar {nombre:objeto}
def define_models(models=dict()):
    # nonlinear models
    models['knn'] = KNeighborsClassifier(n_neighbors=7)
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    
    # ensemble models
    models['bag'] = BaggingClassifier(n_estimators=100)
    models['rf'] = RandomForestClassifier(n_estimators=100)
    models['et'] = ExtraTreesClassifier(n_estimators=100)
    models['gbm'] = GradientBoostingClassifier(n_estimators=100)
    print('Defined %d models' % len(models))
    return models

## 1.3 Evaluación de modelos

Ahora procedemos a crear una función que se encargue de entrenar y evaluar cada algoritmo de clasificación previamente definido en el los datos de entrenamiento y prueba respectivamente. Para este caso usaremos la métrica de `accuracy` para determinar el performance (error) cometido por cada modelo. Como tenemos un diccionario de modelos, generaremos una función que se encargue de iterar sobre cada uno de los modelos y de ejecutar otra función de evaluación. Las funciones serán las siguientes:

In [12]:
# evaluar un solo modelo
def evaluate_model(trainX, trainy, testX, testy, model):
    # Ajustar el modelo
    model.fit(trainX, trainy)
    # Realizar predicciones
    yhat = model.predict(testX)
    # Evaluar predicciones
    accuracy = accuracy_score(testy, yhat)
    return accuracy * 100.0

In [13]:
# Evaluar diccionario de modelos {name:object}, regresa {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
    results = dict()
    for name, model in models.items():
        # Evaluar el modelo
        results[name] = evaluate_model(trainX, trainy, testX, testy, model)
        # Mostrar el proceso
        print('>%s: %.3f' % (name, results[name]))
    return results

## 1.4 Resumen de resultados

Ahora crearemos una función que se encargue de resumir y mostrar todos los resultados obtenidos por cada uno de los modelos ya entrenados. La función será la siguiente:

In [10]:
# Imprimir y mostrar los resultados
def summarize_results(results, maximize=True):
    # Crear lista de (name, mean(scores)) tuplas
    mean_scores = [(k,v) for k,v in results.items()]
    # Ordenar tuplas por score
    mean_scores = sorted(mean_scores, key=lambda x: x[1])
    # reverse para orden descendiente (ej. para accuracy)
    if maximize:
        mean_scores = list(reversed(mean_scores))
    print()
    for name, score in mean_scores:
        print('Name=%s, Score=%.3f' % (name, score))

In [15]:
# Generar el diccionario de modelos
models = define_models()

# Evaluar los modelos
results = evaluate_models(trainX, trainy, testX, testy, models)

# Resumir los resultados de los modelos
summarize_results(results)

Defined 8 models
>knn: 90.329
>cart: 85.612
>svm: 95.046
>bayes: 77.027
>bag: 90.159
>rf: 92.399
>et: 93.926
>gbm: 93.926

Name=svm, Score=95.046
Name=gbm, Score=93.926
Name=et, Score=93.926
Name=rf, Score=92.399
Name=knn, Score=90.329
Name=bag, Score=90.159
Name=cart, Score=85.612
Name=bayes, Score=77.027


De los resultados obtenidos podemos ver que los modelos con la ingeniería de características alcanzaron un performance más allá de lo esperado llegando hasta casi el 95% de precisión.

Ahora vamos a probar el rendimientos de estos algoritmos usando la data sin la ingeniería de características.

# 2. Modelado de datos sin procesar

Recordemos que por defecto los datos que tenemos de total acceleration, body acceleration, and body gyroscope ya se encuentran procesados y en forma tensorial `[samples, timesteps, features]`. Para poder tener estos datos adecuados para nuestros algoritmos de ML necesitamos tenerlos en forma matricial, por lo que procederemos a luego de la lectura de archivos, a transformarlos a su forma matricial usando la función `reshape`. Para este proceso usaremos las funciones que desarrollamos en sesiones previas con unas pequeñas variaciones, como se muestra a continuación:

In [16]:
# Crear función que lee los datos de un archivo en formato txt y los retorna en un dataframe. Están separados por espacios en blanco
def load_file(filename):
    df = pd.read_csv(filename, delim_whitespace=True, header=None)
    return df.values

# Crear función que lee los datos de entrenamiento y de prueba
def load_group(filenames, prefix=''):
    loaded = [load_file(prefix + f) for f in filenames]

    # Apilar los datos en un solo array de 3 dimensiones
    loaded = np.dstack(loaded)
    return loaded

# Crear función que lee los datos de entrenamiento y de prueba
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # cargar los 9 archivos como un único array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_' + group + '.txt', 'total_acc_y_' + group + '.txt',
                  'total_acc_z_' + group + '.txt']
    # body acceleration
    filenames += ['body_acc_x_' + group + '.txt', 'body_acc_y_' + group + '.txt',
                  'body_acc_z_' + group + '.txt']
    # body gyroscope
    filenames += ['body_gyro_x_' + group + '.txt', 'body_gyro_y_' + group + '.txt',
                  'body_gyro_z_' + group + '.txt']
    # cargar input data
    X = load_group(filenames, filepath)
    # cargar class output
    y = load_file(prefix + group + '/y_' + group + '.txt')
    return X, y

In [17]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # Cargo los datos de entrenamiento
    trainX, trainy = load_dataset_group('train', prefix)
    print('datos de entrenamiento', trainX.shape, trainy.shape, sep='\n')
    # Cargo los datos de prueba
    testX, testy = load_dataset_group('test', prefix)
    print('datos de prueba',testX.shape, testy.shape, sep='\n')
    
    # flatten X
    trainX = trainX.reshape((trainX.shape[0], trainX.shape[1] * trainX.shape[2]))
    testX = testX.reshape((testX.shape[0], testX.shape[1] * testX.shape[2]))
    
    # flatten y
    trainy, testy = trainy[:,0], testy[:,0]
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [18]:
# Cargar los datos de entrenamiento y de prueba
trainX, trainy, testX, testy = load_dataset('UCI HAR Dataset/')

datos de entrenamiento
(7352, 128, 9)
(7352, 1)
datos de prueba
(2947, 128, 9)
(2947, 1)
(7352, 1152) (7352,) (2947, 1152) (2947,)


Con estos datos sin ingeniería de características procedemos a probar su performance en esta nueva data.

In [ ]:
# Generar el diccionario de modelos
models = define_models()

# Evaluar los modelos
results = evaluate_models(trainX, trainy, testX, testy, models)

# Resumir los resultados de los modelos
summarize_results(results)

Defined 8 models
>knn: 61.893
>cart: 72.718
>svm: 88.734
>bayes: 72.480
>bag: 84.459
>rf: 84.391
>et: 87.139


# Extensiones

Además de los modelos de ML vistos en esta sesión, hay variaciones que pueden explorarse que pueden mejorar nuestros modelos.

- **Más algoritmos**. Sólo se evaluaron ocho algoritmos de aprendizaje automático en el problema; pruebe con algunos métodos lineales y quizá con otros no lineales y de conjunto y analice sus resultados.
- **Ajuste de algoritmos**. No se realizó ningún ajuste de los algoritmos de aprendizaje automático; se utilizaron principalmente configuraciones predeterminadas. Elija un método como SVM, ExtraTrees o Gradient Boosting y encuentre el conjunto de hiperparámetros óptimo para ver si puede mejorar aún más el rendimiento en el problema.
- **Escalado de datos**. Los datos ya están escalados a [-1,1], quizás por sujeto. Explore si un escalado adicional, como la estandarización, puede resultar en un mejor rendimiento, quizás en métodos sensibles a dicho escalado como kNN.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Oscar David Jaramillo Zuluaga.
</footer>